###  42개의 classes 존재 

In [1]:
import pandas as pd
import pickle
import pprint
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


# %config InlineBackend.figure_format = 'retina'
# !apt -qq -y install fonts-nanum

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

plt.rc('font', family='NanumBarunGothic') 

In [5]:
import pickle
import pprint
from collections import defaultdict
with open('/opt/ml/input/data/label_type.pkl' , 'rb') as f :
    data = pickle.load(f)

data = sorted(data.items() , key = lambda x : data.keys())

str2num_dict = defaultdict()
num2str_dict = defaultdict()

for k, v in data:
    str2num_dict[k] = v
    num2str_dict[v] = k

In [6]:
with open('/opt/ml/input/data/label_type.pkl' , 'rb') as f :
    data = pickle.load(f)

In [7]:
print(data)

{'관계_없음': 0, '인물:배우자': 1, '인물:직업/직함': 2, '단체:모회사': 3, '인물:소속단체': 4, '인물:동료': 5, '단체:별칭': 6, '인물:출신성분/국적': 7, '인물:부모님': 8, '단체:본사_국가': 9, '단체:구성원': 10, '인물:기타_친족': 11, '단체:창립자': 12, '단체:주주': 13, '인물:사망_일시': 14, '단체:상위_단체': 15, '단체:본사_주(도)': 16, '단체:제작': 17, '인물:사망_원인': 18, '인물:출생_도시': 19, '단체:본사_도시': 20, '인물:자녀': 21, '인물:제작': 22, '단체:하위_단체': 23, '인물:별칭': 24, '인물:형제/자매/남매': 25, '인물:출생_국가': 26, '인물:출생_일시': 27, '단체:구성원_수': 28, '단체:자회사': 29, '인물:거주_주(도)': 30, '단체:해산일': 31, '인물:거주_도시': 32, '단체:창립일': 33, '인물:종교': 34, '인물:거주_국가': 35, '인물:용의자': 36, '인물:사망_도시': 37, '단체:정치/종교성향': 38, '인물:학교': 39, '인물:사망_국가': 40, '인물:나이': 41}


In [8]:
# 처음 불러온 tsv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다.
# 변경한 DataFrame 형태는 baseline code description 이미지를 참고해주세요.
def preprocessing_dataset(dataset, label_type):
  label = []
  for i in dataset[8]:
    if i == 'blind':
      label.append(100)
    else:
      label.append(label_type[i])
  out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
  return out_dataset

# tsv 파일을 불러옵니다.
def load_data(dataset_dir):
  # load label_type, classes
  with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)
  # load dataset
  dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
  # preprecessing dataset
  dataset = preprocessing_dataset(dataset, label_type)
  
  return dataset

In [9]:
import pandas as pd 
import seaborn as sns
from transformers import AutoTokenizer

train_dataset = load_data('/opt/ml/input/data/train/train.tsv')
test_dataset = load_data('/opt/ml/input/data/test/test.tsv')

kor_model_name = 'monologg/koelectra-base-v3-discriminator'
multi_model_name = "bert-base-multilingual-cased"
kor_tokenizer = AutoTokenizer.from_pretrained(kor_model_name , do_lower_case = False)
multi_tokenizer = AutoTokenizer.from_pretrained(multi_model_name , do_lower_case = False)

## Entity만 볼까?

### kor : 'monologg/koelectra-base-v3-discriminator'

In [11]:
## kor : 'monologg/koelectra-base-v3-discriminator'

from collections import defaultdict
kor_unknown = set()
kor_label =  defaultdict(int)

for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if '[UNK]' in kor_tokenizer.tokenize(e0):
        kor_unknown.add(e0)
        kor_label[label]+= 1
        print(f"{idx} : {e0} -> {kor_tokenizer.tokenize(e0)}")
        
    if '[UNK]' in kor_tokenizer.tokenize(e1):
        kor_unknown.add(e1)
        kor_label[label]+= 1
        print(f"{idx} : {e1} -> {kor_tokenizer.tokenize(e1)}")

print('---------- 빈도수 확인 ------------')
for k , v in kor_label.items() :
    print(f"label : {num2str_dict[k]} , unk 개수 : {v}")

print(f'---------- 전체 수 : {sum(list(kor_label.values()))}----------------')

4 : 퍼쿤 씨 인트라팃 -> ['퍼', '##쿤', '씨', '[UNK]']
37 : 袁譚 -> ['袁', '[UNK]']
335 : 黃皓 -> ['黃', '[UNK]']
500 : 한예(韓銳) -> ['한', '##예', '(', '韓', '[UNK]', ')']
1372 : 아르툠 미코얀 -> ['[UNK]', '미코', '##얀']
1559 : 森 喜朗 -> ['森', '喜', '[UNK]']
2021 : 北条氏 -> ['北', '[UNK]', '氏']
3765 : 蔚山放送 -> ['[UNK]', '山', '放', '送']
4074 : 됭케르크 -> ['[UNK]']
4291 : 노오(盧塢) -> ['노', '##오', '(', '盧', '[UNK]', ')']
4309 : 李址鎔 -> ['李', '址', '[UNK]']
4334 : Real Madrid Club de Fútbol -> ['Re', '##al', 'Ma', '##d', '##rid', 'Cl', '##ub', 'de', '[UNK]']
4610 : 니콜라이 로바쳅스키 -> ['니콜라', '##이', '[UNK]']
4878 : 로퀜스 -> ['[UNK]']
5448 : 竇武 -> ['[UNK]', '武']
5459 : 丸彫 -> ['丸', '[UNK]']
7799 : 張道斌 -> ['張', '道', '[UNK]']
8424 : 刘骅骐骥 -> ['[UNK]', '[UNK]', '[UNK]', '[UNK]']
---------- 빈도수 확인 ------------
label : 인물:부모님 , unk 개수 : 1
label : 관계_없음 , unk 개수 : 6
label : 인물:별칭 , unk 개수 : 5
label : 인물:제작 , unk 개수 : 1
label : 단체:별칭 , unk 개수 : 3
label : 인물:기타_친족 , unk 개수 : 1
label : 인물:출신성분/국적 , unk 개수 : 1
---------- 전체 수 : 18----------------


In [ ]:
['인트라팃' , '譚' , '皓' , '銳' , '아르툠', '朗' , '条' ,'蔚' , '됭케르크' , '塢']

In [19]:
print(kor_tokenizer.tokenize('현재 진도군은 ▲안전하고 행복한 진도 ▲잘사는 진도 ▲지속가능한 진도 ▲품격 있고 활력 있는 진도 ▲아름답고 살기 좋은 진도 등 5개 분야로 공약사항을 구분 점검‧실천 하고 있다.'))
print(multi_tokenizer.tokenize('현재 진도군은 ▲안전하고 행복한 진도 ▲잘사는 진도 ▲지속가능한 진도 ▲품격 있고 활력 있는 진도 ▲아름답고 살기 좋은 진도 등 5개 분야로 공약사항을 구분 점검‧실천 하고 있다. '))

['현재', '진도', '##군', '##은', '▲', '##안전', '##하', '##고', '행복', '##한', '진도', '▲', '##잘', '##사', '##는', '진도', '▲', '##지', '##속', '##가', '##능', '##한', '진도', '▲', '##품', '##격', '있', '##고', '활력', '있', '##는', '진도', '▲', '##아', '##름', '##답', '##고', '살', '##기', '좋', '##은', '진도', '등', '5', '##개', '분야', '##로', '공약', '##사항', '##을', '구분', '점검', '‧', '실천', '하고', '있', '##다', '.']
['▁현재', '▁진', '도', '군', '은', '▁▲', '안전', '하고', '▁행복한', '▁진', '도', '▁▲', '잘', '사는', '▁진', '도', '▁▲', '지', '속', '가능', '한', '▁진', '도', '▁▲', '품', '격', '▁있고', '▁', '활', '력', '▁있는', '▁진', '도', '▁▲', '아', '름', '답', '고', '▁살', '기', '▁좋은', '▁진', '도', '▁등', '▁5', '개', '▁분야', '로', '▁공', '약', '사항', '을', '▁구분', '▁', '점검', '‧', '실', '천', '▁하고', '▁있다', '.']


### multi : "bert-base-multilingual-cased"

In [12]:
multi_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased" , do_lower_case = False)
from collections import defaultdict
multi_unknown = set()
multi_label = defaultdict(int)

for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if  '[UNK]' in multi_tokenizer.tokenize(e0):
        multi_unknown.add(e0)
        multi_label[label]+= 1
        print(f"{idx} : {e0} -> {multi_tokenizer.tokenize(e0)}")

    if '[UNK]' in multi_tokenizer.tokenize(e1):
        multi_unknown.add(e1)
        multi_label[label]+= 1
        print(f"{idx} : {e1} -> {multi_tokenizer.tokenize(e1)}")
        
print('---------- 빈도수 확인 ------------')
for k , v in multi_label.items() :
    print(f"label : {num2str_dict[k]} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(multi_label.values()))}----------------')

4 : 퍼쿤 씨 인트라팃 -> ['퍼', '##쿤', '씨', '[UNK]']
169 : 윰댕 -> ['[UNK]']
170 : 갸넨드라 -> ['[UNK]']
239 : 개똥벌레 -> ['[UNK]']
300 : 잣나무 -> ['[UNK]']
361 : 0 – 1 -> ['0', '[UNK]', '1']
374 : ㈜빵굽는사람들 -> ['[UNK]']
390 : 쥘 베른 -> ['[UNK]', '베', '##른']
522 : 크리스털 팰리스 -> ['크', '##리스', '##털', '[UNK]']
523 : 윰댕 -> ['[UNK]']
582 : 맬컴 -> ['[UNK]']
808 : 드림웍스 -> ['[UNK]']
955 : ☎ -> ['[UNK]']
1026 : ㈜로우카본테크 -> ['[UNK]']
1084 : ㈜진성퍼니처 -> ['[UNK]']
1356 : 펄 잼 -> ['펄', '[UNK]']
1372 : 아르툠 미코얀 -> ['[UNK]', '미', '##코', '##얀']
1452 : 세이코 엡손 -> ['세', '##이', '##코', '[UNK]']
1521 : 원흠 -> ['[UNK]']
1563 : 웅진씽크빅 -> ['[UNK]']
1585 : 이궈달라 -> ['[UNK]']
1753 : ㈜호원 -> ['[UNK]']
1753 : ㈜드림씨엔지 -> ['[UNK]']
1862 : 제임스 헷필드 -> ['제', '##임', '##스', '[UNK]']
1881 : 한땀봉사단 -> ['[UNK]']
1906 : ㈜광주글로벌모터스 -> ['[UNK]']
1977 : 쥘 페리 -> ['[UNK]', '페', '##리']
2111 : 응우옌반냑 -> ['[UNK]']
2133 : 1912년 1월 9일 – 2000년 11월 20일 -> ['1912', '##년', '1월', '9일', '[UNK]', '2000년', '11월', '20일']
2207 : 뽀뽀뽀 -> ['[UNK]']
2212 : 개똥벌레 -> ['[UNK]']
2343 : 투게더광산나

In [28]:
multi_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large' , do_lower_case = False)
multi_tokenizer.unk_token

'<unk>'

In [29]:

multi_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large' , do_lower_case = False)
from collections import defaultdict
multi_unknown = set()
multi_label = defaultdict(int)

for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if  '<unk>' in multi_tokenizer.tokenize(e0):
        multi_unknown.add(e0)
        multi_label[label]+= 1
        print(f"{idx} : {e0} -> {multi_tokenizer.tokenize(e0)}")

    if '<unk>' in multi_tokenizer.tokenize(e1):
        multi_unknown.add(e1)
        multi_label[label]+= 1
        print(f"{idx} : {e1} -> {multi_tokenizer.tokenize(e1)}")
        
print('---------- 빈도수 확인 ------------')
for k , v in multi_label.items() :
    print(f"label : {num2str_dict[k]} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(multi_label.values()))}----------------')

---------- 빈도수 확인 ------------
---------- 전체 수 : 0----------------


### K-tokenizer를 바꾸면 ? 

In [ ]:
kor_model_name = 'kykim/bert-kor-base'
kor_tokenizer = AutoTokenizer.from_pretrained(kor_model_name , do_lower_case = False)

In [ ]:
## kor : 'kykim/bert-kor-base'

from collections import defaultdict
kor_unknown = set()
kor_label =  defaultdict(int)

for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if '[UNK]' in kor_tokenizer.tokenize(e0):
        kor_unknown.add(e0)
        kor_label[label]+= 1
        print(f"{idx} : {e0} -> {kor_tokenizer.tokenize(e0)}")
        
    if '[UNK]' in kor_tokenizer.tokenize(e1):
        kor_unknown.add(e1)
        kor_label[label]+= 1
        print(f"{idx} : {e1} -> {kor_tokenizer.tokenize(e1)}")

print('---------- 빈도수 확인 ------------')
for k , v in kor_label.items() :
    print(f"label : {num2str_dict[k]} , unk 개수 : {v}")

print(f'---------- 전체 수 : {sum(list(kor_label.values()))}----------------')

In [ ]:
test_dataset

In [ ]:
kor_tokenizer1 = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator', do_lower_case = False)
kor_tokenizer2 = AutoTokenizer.from_pretrained('kykim/bert-kor-base' , do_lower_case = False)
multi_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased" , do_lower_case = False)

kor_label1 = 0
kor_label2 = 0
multi_label = 0

for idx, rows in test_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if '[UNK]' in kor_tokenizer1.tokenize(e0):
        kor_label1+= 1
        
    if '[UNK]' in kor_tokenizer1.tokenize(e1):
        kor_label1+= 1

        
    if '[UNK]' in multi_tokenizer.tokenize(e0):
        multi_label += 1
        
    if '[UNK]' in multi_tokenizer.tokenize(e1):
        multi_label+= 1  
        
        
    if '[UNK]' in kor_tokenizer2.tokenize(e0):
        kor_label2+= 1
        
    if '[UNK]' in kor_tokenizer2.tokenize(e1):
        kor_label2+= 1
      
        

print(f'----------koelectra 전체 수 : {kor_label1}----------------')
print(f'----------multilingual 전체 수 : {multi_label}----------------')
print(f'----------kor-base 전체 수 : {kor_label2}----------------')

# 결론
- Entity만 봤을 땐 , multilingual의 UNK가 더 적다.
- K-Tokenizer를 사용하고 싶다면 koelectra 추천! 

-----------------------------------------------------------------------------
# 번외편 
label 말고 단체, 인물, 사망 일시 이런 거 중에 unk가 뭐가 많은 지 궁금한데? 

In [ ]:
## kor : 'monologg/koelectra-base-v3-discriminator'

from collections import defaultdict
kor_tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator' , do_lower_case = False , handle_chinese_chars=True)
kor_unknown = set()
kor_label =  defaultdict(int)


for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if '[UNK]' in kor_tokenizer.tokenize(e0):
        kor_unknown.add(e0)
        print(f"{num2str_dict[label]} | {idx} : {e0} -> {kor_tokenizer.tokenize(e0)}")
        
        if label == 0 :
            kor_label[num2str_dict[label]]+= 1
        else : 
            str_label = num2str_dict[label].split(':')[0]
            kor_label[str_label]+= 1

        
    if '[UNK]' in kor_tokenizer.tokenize(e1):
        kor_unknown.add(e1)
        print(f"{num2str_dict[label]} | {idx} : {e1} -> {kor_tokenizer.tokenize(e1)}")
        
        if label == 0 :
            kor_label[num2str_dict[label]]+= 1
        else : 
            str_label = num2str_dict[label].split(':')[1]
            kor_label[str_label]+= 1

print('---------- 빈도수 확인 ------------')
for k , v in kor_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(kor_label.values()))}----------------')

In [ ]:
from collections import defaultdict
multi_unknown = set()
multi_label = defaultdict(int)

for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if '[UNK]' in multi_tokenizer.tokenize(e0):
        multi_unknown.add(e0)
        print(f"{num2str_dict[label]} | {idx} : {e0} -> {multi_tokenizer.tokenize(e0)}")
        
        if label == 0 :
            multi_label[num2str_dict[label]]+= 1
        else : 
            str_label = num2str_dict[label].split(':')[0]
            multi_label[str_label]+= 1

    if '[UNK]' in multi_tokenizer.tokenize(e1):
        multi_unknown.add(e1)
        print(f"{num2str_dict[label]} | {idx} : {e1} -> {multi_tokenizer.tokenize(e1)}")
        
        if label == 0 :
            multi_label[num2str_dict[label]]+= 1
        else : 
            str_label = num2str_dict[label].split(':')[1]
            multi_label[str_label]+= 1
        

print('---------- 빈도수 확인 ------------')
for k , v in multi_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(multi_label.values()))}----------------')

In [ ]:
## kor : 'kykim/bert-kor-base'

from collections import defaultdict
kor_tokenizer = AutoTokenizer.from_pretrained('kykim/bert-kor-base', do_lower_case = False)
kor_unknown = set()
kor_label =  defaultdict(int)

for idx, rows in train_dataset.iterrows():
    e0 , e1 , label = rows[1] , rows[2] , rows[3]
    
    if '[UNK]' in kor_tokenizer.tokenize(e0):
        kor_unknown.add(e0)
        print(f"{num2str_dict[label]} | {idx} : {e0} -> {kor_tokenizer.tokenize(e0)}")
        
        if label == 0 :
            kor_label[num2str_dict[label]]+= 1
        else : 
            str_label = num2str_dict[label].split(':')[0]
            kor_label[str_label]+= 1

        
    if '[UNK]' in kor_tokenizer.tokenize(e1):
        kor_unknown.add(e1)
        print(f"{num2str_dict[label]} | {idx} : {e1} -> {kor_tokenizer.tokenize(e1)}")
        
        if label == 0 :
            kor_label[num2str_dict[label]]+= 1
        else : 
            str_label = num2str_dict[label].split(':')[1]
            kor_label[str_label]+= 1

print('---------- 빈도수 확인 ------------')
for k , v in kor_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(kor_label.values()))}----------------')

## sentence에 대해서 볼까?

In [20]:
## kor : 'monologg/koelectra-base-v3-discriminator'
import pprint
from collections import defaultdict
kor_tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator' , do_lower_case = False , handle_chinese_chars=True)
kor_label =  defaultdict(int)


for idx, rows in train_dataset.iterrows():
    sentence , label = rows[0], rows[3]
    
    if '[UNK]' in kor_tokenizer.tokenize(sentence):
        pprint.pprint(f"{num2str_dict[label]} | {idx} : {sentence} -> {kor_tokenizer.tokenize(sentence)}")
        kor_label[num2str_dict[label]]+=1
        
print('---------- 빈도수 확인 ------------')
for k , v in kor_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(kor_label.values()))}----------------')

('인물:부모님 | 4 : 람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 부인 사이의 셋째 아들로 '
 "태어났다. -> ['람', '##캄', '##행', '왕', '##은', '123', '##7', '##년', '##에', '##서', "
 "'124', '##7', '##년', '사이', '수', '##코', '##타이', '##의', '왕', '퍼', '##쿤', '씨', "
 "'[UNK]', '쓰', '##엉', '부인', '사이', '##의', '셋째', '아들', '##로', '태어났', '##다', "
 "'.']")
('인물:형제/자매/남매 | 35 : 이의방은 문극겸과 친한데다 또 자기 동생 이린(李隣)이 문극겸의 사위였으므로 그 말을 따랐다. -> '
 "['이의', '##방', '##은', '문', '##극', '##겸', '##과', '친한', '##데', '##다', '또', "
 "'자기', '동생', '이', '##린', '(', '李', '[UNK]', ')', '이', '문', '##극', '##겸', "
 "'##의', '사위', '##였', '##으므로', '그', '말', '##을', '따랐', '##다', '.']")
('관계_없음 | 37 : 이부는 원상과 헤어지게 되었으므로 어쩔 수 없이 원담(袁譚)에게 항복했고 평원(平原)에 부임하였다. -> '
 "['이부', '##는', '원상', '##과', '헤어지', '##게', '되', '##었', '##으므로', '어쩔', '수', "
 "'없이', '원', '##담', '(', '袁', '[UNK]', ')', '에게', '항복', '##했', '##고', '평원', "
 "'(', '平', '原', ')', '에', '부임', '##하', '##였', '##다', '.']")
("관계_없음 | 61 : 천계제 사후에 숭정제(崇禎帝)는 황후 장씨에게 존호를 가상하여, 황태후 급으로 정성스럽게 모셨다. -> ['천', "
 "'##

In [30]:
import pprint
from collections import defaultdict

multi_label = defaultdict(int)

for idx, rows in train_dataset.iterrows():
    sentence , label = rows[0], rows[3]
    
    if '<unk>' in multi_tokenizer.tokenize(sentence):
       # pprint.pprint(f"{num2str_dict[label]} | {idx} : {sentence} -> {multi_tokenizer.tokenize(sentence)}")
        multi_label[num2str_dict[label]]+=1
        
        
print('---------- 빈도수 확인 ------------')
for k , v in multi_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(multi_label.values()))}----------------')

---------- 빈도수 확인 ------------
---------- 전체 수 : 0----------------


In [ ]:
## kor : 'kykim/bert-kor-base'
import pprint
from collections import defaultdict
kor_tokenizer = AutoTokenizer.from_pretrained('kykim/bert-kor-base' , do_lower_case = False , handle_chinese_chars=True)
kor_label =  defaultdict(int)


for idx, rows in train_dataset.iterrows():
    sentence , label = rows[0], rows[3]
    
    if '[UNK]' in kor_tokenizer.tokenize(sentence):
#         pprint.pprint(f"{num2str_dict[label]} | {idx} : {sentence} -> {kor_tokenizer.tokenize(sentence)}")
        kor_label[num2str_dict[label]]+=1
        
print('---------- 빈도수 확인 ------------')
for k , v in kor_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(kor_label.values()))}----------------')

In [14]:
import pprint
from collections import defaultdict
multi_label = defaultdict(int)


for idx, rows in train_dataset.iterrows():
    sentence , label = rows[0], rows[3]
    
    if '[UNK]' in multi_tokenizer.tokenize(sentence):
        pprint.pprint(f"{num2str_dict[label]} | {idx} : {sentence} -> {multi_tokenizer.tokenize(sentence)}")
        multi_label[num2str_dict[label]]+=1
        
print('---------- 빈도수 확인 ------------')
for k , v in multi_label.items() :
    print(f"label : {k} , unk 개수 : {v}")
    
print(f'---------- 전체 수 : {sum(list(multi_label.values()))}----------------')


---------- 빈도수 확인 ------------
---------- 전체 수 : 0----------------


In [ ]:
# 한글 폰트 적용
plt.rc('font', family='NanumBarunGothic') 
# 캔버스 사이즈 적용
plt.rcParams["figure.figsize"] = (10,8)
multi_df = pd.DataFrame()
multi_df['label'] = [str2num_dict[k] for k in multi_label.keys()]
multi_df['cnt'] = [v for v in multi_label.values()]
multi_df.sort_values(by=['label'], axis=0 , inplace = True)
# multi_df.set_index('label',inplace = True)
multi_df
sns.countplot(x = 'cnt' , data = multi_df)

## 문장 길이의 분포 확인하기

In [3]:
# 한글 폰트 적용
plt.rc('font', family='NanumBarunGothic') 
# 캔버스 사이즈 적용
plt.rcParams["figure.figsize"] = (10,10)

sns.displot([len(s) for s in train_dataset['sentence']] , bins = 50)
ax = plt.axvline(x=190, color='g', linestyle='--', linewidth=1)
ax = plt.axvline(x=278, color='b', linestyle='--', linewidth=1)
plt.show()

NameError: name 'sns' is not defined

- 95% 이상을 outlier로 잡으면 너무 많다. => 450개를 삭제 해야함 
- 99% 이상을 outlier로 잡으면 적당한가,,? 잘 모르겠다,,? => 90개 삭제
- max_len  = 256으로 잡으면 대략 130개 삭제 해야함 

- ❤ 99% 이상을 outlier라고 가정해보자. 
- max_length = 278일 때, entity들의 분포는 어떻게 되는 지 
- max_length = 278일 때, outlier들의 분포는 어떻게 되는지

In [ ]:
raw_train_dataset = dataset = pd.read_csv('./input/data/train/train.tsv', delimiter='\t', header=None)
raw_train_dataset.head()

In [ ]:
# 기준은 entity가 끝나는 지점 
# 한글 폰트 적용
plt.rc('font', family='NanumBarunGothic') 
# 캔버스 사이즈 적용
plt.rcParams["figure.figsize"] = (10,10)

entity_len = [s for s in raw_train_dataset[4]]
entity_len.extend([s for s in raw_train_dataset[7]])
sns.displot(entity_len , bins = 50)
ax = plt.axvline(x=278, color='b', linestyle='--', linewidth=1)
plt.show()

In [ ]:
raw_train_dataset

In [ ]:
# 90%, 95%, 99% 지점의 값 구하기
label_dict = defaultdict(int)
cnt = 0
for row in range(9000):
    if raw_train_dataset.iloc[row][4] > 278 or raw_train_dataset.iloc[row][7] > 278 :
        label_dict[raw_train_dataset.iloc[row][8]] += 1
        cnt += 1

print(label_dict)
print(cnt)

- 278을 max_len 지정하게 되면! 잘려나가는 entity는 12개가 되는데 
    - '단체:상위_단체': 1, 
    - '관계_없음': 5, 
    - '단체:별칭': 5,
    - '인물:직업/직함': 1
- 모두 class가 적은 편은 아님 제일 적은 인물:별칭은 103개 정도이다.  => 괜찮을지도?     

In [ ]:
# 90%, 95%, 99% 지점의 값 구하기
label_dict = defaultdict(int)
cnt = 0
for row in range(9000):
    if raw_train_dataset.iloc[row][4] > 190 or raw_train_dataset.iloc[row][7] > 190 :
        label_dict[raw_train_dataset.iloc[row][8]] += 1
        cnt += 1

print(label_dict)
print(cnt)

- 190을 max_len 지정하게 되면! 잘려나가는 entity는 54개가 되는데 
    - '단체:상위_단체': 4, '관계_없음': 23, '단체:모회사': 2, '인물:종교': 1, '단체:별칭': 5, '인물:소속단체': 7, '인물:직업/직함': 2, '인물:부모님': 1, '단체:본사_도시': 2, '인물:자녀': 1, '인물:배우자': 1, '단체:구성원_수': 1, '인물:출신성분/국적': 2, '단체:하위_단체': 1, '단체:구성원': 1}
    
- 단체:구성원_수은 9개 정도이다. => 8개나 9개나 똑같나? 적은 건 살려야하지않나?ㅜㅜ  

In [40]:
import pandas as pd 
train_dataset = pd.read_csv('/opt/ml/input/data/train/train.tsv' , delimiter = '\t' , header = None)

In [ ]:
df = pd.read_csv('/opt/ml/input/data/train/train.tsv' , delimiter = '\t' , header = None)

f = open('/opt/ml/input/data/train/modify.txt' , 'r')
modi = []
while True:
    line = f.readline()
    if not line :
        break
    line = line.strip()
    id = line.split(' ')[0]
    label = line.split(' ')[1]
    modi.append([id, label])
    
for id, label in modi:
    print(f"전 : {df[df[0] == id][8]}")
    df.loc[df[0]==id , 8] = label        
    print(f"후 : {df[df[0] == id][8]}")
    
df.to_csv('/opt/ml/input/data/train/train.tsv' , index = False , header = None , sep = '\t')

In [ ]:
df = pd.read_csv('/opt/ml/input/data/train/train.tsv' , delimiter = '\t' , header = None)
df.head()

In [ ]:
for id, label in modi:
    print(f"전 : {df[df[0] == id][8]}")
    df.loc[df[0]==id , 8] = label        
    print(f"후 : {df[df[0] == id][8]}")
    
df.to_csv('/opt/ml/input/data/train/train.tsv' , index = False , header = None , sep = '\t')

In [ ]:
df.to_csv('/opt/ml/input/data/train/train.tsv' , index = False , header = None , sep = '\t')

In [42]:
train_dataset[1][:10]

0    영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...
1    선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...
2    유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...
3    용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...
4    람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...
5    이와 관련해 AFCD는 대학 소속 전문가와 세계동물보건기구(OIE)가 사람과 동물 ...
6    신형 K5 사전계약 돌풍에 대해 기아자동차 관계자는 "한눈에 각인될 정도의 강렬한 ...
7    2008년 이전 팀명칭은 나고야 그램퍼스 에이트였고 에이트는 나고야 시 표장에 있는...
8    SW융합교육원 조영주 담당교수는 “4차 산업혁명에 따라 SW역량은 전문가뿐만 아니라...
9    김동양 NH투자증권 연구원은 “삼성SDS의 3분기 매출액은 2조6604억원, 영업이...
Name: 1, dtype: object